In [ ]:
# Copyright 2026 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# AI-Powered Custom Location Scoring with Places Insights & Gemini

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/googlemaps-samples/insights-samples/blob/main/places_insights/notebooks/custom_location_scores/places_insights_custom_location_scores.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2Fgooglemaps-samples%2Finsights-samples%2Fmain%2Fplaces_insights%2Fnotebooks%2Fcustom_location_scores%2Fplaces_insights_custom_location_scores.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/bigquery/import?url=https://github.com/googlemaps-samples/insights-samples/blob/main/places_insights/notebooks/custom_location_scores/places_insights_custom_location_scores.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/bigquery/v1/32px.svg" alt="BigQuery Studio logo"><br> Open in BigQuery Studio
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/googlemaps-samples/insights-samples/blob/main/places_insights/notebooks/custom_location_scores/places_insights_custom_location_scores.ipynb">
      <img width="32px" src="https://upload.wikimedia.org/wikipedia/commons/9/91/Octicons-mark-github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

### Overall Goal

This notebook demonstrates a powerful workflow for generating **Custom Location Scores** ("suitability scores") for real estate or travel. Instead of relying solely on raw counts of nearby amenities, we combine **quantitative data** (Google Maps Platform Public Datasets) with **qualitative data** (listing descriptions) and use Generative AI to calculate a nuanced score for a specific user persona (e.g., a "Young Family").

The result is an **interactive map with color-coded markers**, where the AI not only rates a location (0-10) but provides the **reasoning** behind the score directly in the map popup.

### Key Technologies Used

*   **[Places Insights](https://developers.google.com/maps/documentation/placesinsights):** A Google Maps Platform dataset available in BigQuery that allows for advanced statistical analysis. It provides aggregated counts and density information for Places-of-Interest (POI).
*   **[BigQuery](https://cloud.google.com/bigquery):** The serverless data warehouse used to store the listings and execute the analysis.
*   **[BigQuery AI.GENERATE](https://docs.cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-ai-generate):** A SQL function that allows us to call Generative AI models (like Gemini) directly within a query to generate scores and text reasoning.
*   **[Vertex AI (Gemini)](https://cloud.google.com/vertex-ai/):** We use the `gemini-flash-latest` model to analyze the combined data.
*   **[Google Maps 2D Tiles](https://developers.google.com/maps/documentation/tile/2d-tiles-overview):** To use Google Maps as the high-resolution basemap for visualization.
*   **Python Libraries:**
    * **[Folium](https://python-visualization.github.io/folium/latest/)** for creating the final interactive, layered map.
    * **[Pandas](https://pandas.pydata.org/)** for handling the synthetic listing data.

See [Google Maps Platform Pricing](https://mapsplatform.google.com/intl/en_uk/pricing/) and [BigQuery Pricing](https://cloud.google.com/bigquery/pricing) for costs associated with running this notebook.

**Note:** The `AI.GENERATE` function passes input to a Gemini model and incurs charges in Vertex AI each time it is called. For details on managing these costs, see the **[Best Practices](https://docs.cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-ai-generate#best_practices)** section of the documentation.

### The Step-by-Step Workflow

1.  **Generate Target Candidates:** We begin by creating a dataset of hypothetical apartment listings (including descriptions like "Quiet street" vs. "Busy intersection"). **To store this data, the notebook automatically creates a dedicated BigQuery dataset in your project.**

2.  **The AI-SQL Scoring Engine:** We execute a single, powerful SQL query in BigQuery that:
    *   Performs a spatial join to count specific amenities (Parks, Schools, Museums) within 800m of each listing.
    *   Constructs a natural language prompt combining these counts with the listing description.
    *   Calls **Gemini** via `AI.GENERATE` to calculate a "Family-Friendliness Score" and explain *why* that score was given.

3.  **Visual Analysis:** In the final step, we render the scored locations on a Google Map. We use **color-coded pins** (Green=High Score, Red=Low Score) to instantly identify the best candidates. Clicking a marker reveals a popup with the AI's detailed reasoning.

4.  **Resource Cleanup:** A final, interactive cell allows you to delete the temporary BigQuery dataset and all its contents, ensuring you leave your project clean.

### **How to Use This Notebook**

1.  **Set Up Secrets:** Before you begin, you must configure two secrets in the Colab "Secrets" tab (the **key icon** on the left menu):
    *   `GCP_PROJECT_ID`: Your Google Cloud Project ID.
    *   `GMP_API_KEY`: Your Google Maps Platform API key.

2.  **Enable APIs:** Ensure the following APIs are enabled in your Google Cloud Console for the project:
    *   **Maps Tiles API** (for the visualization).
    *   **Vertex AI API** (for the AI scoring).

3.  **Subscribe to Places Insights (US Data):**
    This notebook analyzes locations in New York, so you must have access to the United States dataset in BigQuery.
    *   **Sign Up:** If you haven't already, [sign up for access](https://developers.google.com/maps/documentation/placesinsights/cloud-setup) to Places Insights.
    *   **Choose Your Dataset:**
        *   **Full Data Option:** Subscribe to the full **"Places Insights - US"** listing. Ensure your linked dataset is named `places_insights___us`.
        *   **Sample Data Option:** You may also use the **[US Sample Data](https://developers.google.com/maps/documentation/placesinsights/cloud-setup#sample_data)** ("Places Insights - US - sample"), as it covers New York City.
    *   **Important:** If you use the **Sample Data** (or name your dataset differently), you **must update** the `PLACES_TABLE` variable in **Phase 3** of this notebook to match your linked dataset name (e.g., `places_insights___us___sample`).

4.  **Run the Cells:** Once the secrets and data subscriptions are ready, simply run the cells in order.

In [ ]:
# @title Phase 1: Authentication & Setup
# @markdown This cell authenticates you to Google Cloud, retrieves API keys, and **creates a new BigQuery dataset** (`places_insights_custom_score_demo`) in your project to store the demo data.

import sys
from google.colab import userdata, auth
from google.cloud import bigquery
import google.auth
import pandas as pd

# 1. Retrieve Secrets
try:
    GCP_PROJECT_ID = userdata.get('GCP_PROJECT_ID')
    GMP_API_KEY = userdata.get('GMP_API_KEY')
    print(f"✅ Secrets retrieved for project: {GCP_PROJECT_ID}")
except userdata.SecretNotFoundError as e:
    raise ValueError("Missing Secrets! Please add 'GCP_PROJECT' and 'GMP_API_KEY' to Colab Secrets.") from e

# 2. Authenticate User
auth.authenticate_user(project_id=GCP_PROJECT_ID)
print("✅ User Authenticated.")

# 3. Initialize BigQuery Client
client = bigquery.Client(project=GCP_PROJECT_ID)
REGION = "US" # Places Insights is in the US multi-region
DATASET_ID = f"{GCP_PROJECT_ID}.places_insights_custom_score_demo"

# Create a working dataset to store our staging table
ds = bigquery.Dataset(DATASET_ID)
ds.location = REGION
client.create_dataset(ds, exists_ok=True)
print(f"✅ Working dataset ready: {DATASET_ID}")

In [ ]:
# @title Phase 2: Create Target Listings Data
# @markdown Generates a hypothetical portfolio of **candidate properties** to simulate a real-world scenario.
# @markdown
# @markdown Each listing combines **geospatial coordinates** (for spatial analysis) with **text descriptions** (for AI sentiment analysis), staging the data in **BigQuery** for the scoring engine.

# 1. Define Apartment Listings (Quantitative + Qualitative)
data = [
    {
        "id": 1,
        "name": "The Midtown Lofts",
        "lat": 40.7511, "lng": -73.9846, # Times Square area
        "description": "Modern loft in the heart of the city. High energy, constant noise, and nightlife right downstairs. Great for singles, difficult for quiet sleepers."
    },
    {
        "id": 2,
        "name": "Suburban Garden Suites",
        "lat": 40.6586, "lng": -73.9764, # Near Prospect Park
        "description": "Quiet, tree-lined street near a major park. Large backyard, very low traffic. Perfect for pets and kids."
    },
    {
        "id": 3,
        "name": "The Transit Hub",
        "lat": 40.6892, "lng": -73.9772, # Near Atlantic Terminal
        "description": "Unbeatable commute. Located directly above a major subway hub. Extremely busy intersection, no green space immediately adjacent."
    },
    {
        "id": 4,
        "name": "Riverside Retreat",
        "lat": 40.8000, "lng": -73.9700, # Upper West Side / Riverside
        "description": "Scenic views of the river. historic building, slower pace of life but plenty of cultural amenities and museums nearby."
    },
    {
        "id": 5,
        "name": "Industrial Fixer-Upper",
        "lat": 40.7050, "lng": -73.9200, # Bushwick area
        "description": "Converted warehouse space. Very gritty, few amenities nearby, but huge open floor plan. Developing neighborhood."
    }
]

df_listings = pd.DataFrame(data)

# 2. Upload to BigQuery
table_id = f"{DATASET_ID}.apartment_listings"

job_config = bigquery.LoadJobConfig(
    # specific schema to ensure Lat/Lng are floats
    schema=[
        bigquery.SchemaField("id", "INTEGER"),
        bigquery.SchemaField("name", "STRING"),
        bigquery.SchemaField("lat", "FLOAT"),
        bigquery.SchemaField("lng", "FLOAT"),
        bigquery.SchemaField("description", "STRING"),
    ],
    write_disposition="WRITE_TRUNCATE", # Overwrite if exists
)

job = client.load_table_from_dataframe(df_listings, table_id, job_config=job_config)
job.result() # Wait for job to complete

print(f"✅ Data uploaded to `{table_id}`")
print(f"   Rows inserted: {job.output_rows}")
df_listings.head()

In [ ]:
# @title Phase 3: Custom Location Scoring Engine
# @markdown Executes the core **AI-SQL Scoring Engine** by fusing **Places Insights** data with **BigQuery `AI.GENERATE`**.
# @markdown
# @markdown We first perform a spatial join to quantify nearby amenities (parks, schools, etc.), then pass these metrics, along with listing descriptions, directly into **Gemini** to calculate a nuanced Family-Friendliness Score and explanation.

from google.cloud import bigquery
import pandas as pd

# 1. Define Resources
PLACES_TABLE = f"{GCP_PROJECT_ID}.places_insights___us.places"
LISTINGS_TABLE = f"{DATASET_ID}.apartment_listings"

# 2. Construct the Global Endpoint URL
# This explicit URL ensures we target the global publisher model directly
MODEL_NAME = "gemini-flash-latest"
ENDPOINT_URL = f"https://aiplatform.googleapis.com/v1/projects/{GCP_PROJECT_ID}/locations/global/publishers/google/models/{MODEL_NAME}"

sql_query = f"""
WITH insight_counts AS (
  -- Mandatory Privacy Clause
  SELECT WITH AGGREGATION_THRESHOLD
    listings.id,
    listings.name,
    listings.lat,
    listings.lng,
    listings.description,
    -- Quantitative Metrics (COALESCE handles privacy NULLs later)
    COUNTIF(places.primary_type = 'park') AS park_count,
    COUNTIF(places.primary_type = 'museum') AS museum_count,
    COUNTIF(places.primary_type = 'restaurant' AND places.good_for_children = TRUE) AS family_restaurant_count,
    COUNTIF(places.primary_type IN ('primary_school', 'secondary_school')) AS school_count
  FROM
    `{LISTINGS_TABLE}` AS listings
  LEFT JOIN
    `{PLACES_TABLE}` AS places
    ON ST_DWITHIN(ST_GEOGPOINT(listings.lng, listings.lat), places.point, 800)
  GROUP BY
    listings.id, listings.name, listings.lat, listings.lng, listings.description
),

prepared_prompts AS (
  SELECT
    *,
    -- Prompt Engineering with Null Handling
    FORMAT('''
      You are an expert real estate analyst. Generate a 'Family-Friendliness Score' (0-10) for this location.

      Target User: Young family with a toddler, looking for safety, green space, and convenience.

      Location Data:
      - Description: %s
      - Parks nearby (800m): %d
      - Schools (Primary/Secondary) nearby: %d
      - Museums nearby: %d
      - Family-friendly restaurants: %d

      Scoring Rules:
      - High importance: High park count and presence of schools.
      - Negative modifiers: Descriptions mentioning "noise", "nightlife", or "industrial".
      - Positive modifiers: Descriptions mentioning "quiet", "backyard", or "community".

      Output only the score and a 1-sentence reasoning.
    ''',
      description,
      COALESCE(park_count, 0),
      COALESCE(school_count, 0),
      COALESCE(museum_count, 0),
      COALESCE(family_restaurant_count, 0)
    ) AS prompt_text
  FROM insight_counts
)

-- Final Step: Call AI.GENERATE with Explicit Endpoint
SELECT
  id,
  name,
  lat,
  lng,
  generated_output.family_friendliness_score AS score,
  generated_output.reasoning
FROM (
  SELECT
    *,
    AI.GENERATE(
      prompt_text,
      endpoint => '{ENDPOINT_URL}',
      output_schema => 'family_friendliness_score FLOAT64, reasoning STRING'
    ) AS generated_output
  FROM prepared_prompts
)
ORDER BY score DESC;
"""

print(f"🧠 Running AI Scoring Query...")
print(f"   Target Model: {MODEL_NAME}")

try:
    query_job = client.query(sql_query)
    df_scored = query_job.to_dataframe()
    print("✅ Scoring Complete!")

    # Display results
    pd.set_option('display.max_colwidth', None)
    display(df_scored[['name', 'score', 'reasoning']])

except Exception as e:
    print(f"❌ Query Failed. Error: {e}")

In [ ]:
# @title Phase 4a: Maps Backend Initialization (Session, Copyright & Assets)
# @markdown This cell manages the Google Maps 2D Tiles API handshake. It performs the following steps:
# @markdown 1.  **Session Creation:** Authenticates and requests a "Roadmap" session for the target region.
# @markdown 2.  **Attribution Fetching:** Queries the API for the specific copyright text required for the configured viewport.
# @markdown 3.  **Asset Preparation:** Generates the compliant HTML for the Google Maps logo overlay.

import requests

# 1. Validation & Data Bounds
if 'df_scored' not in locals() or df_scored.empty:
    raise ValueError("No data found! Please run Phase 3 successfully first.")
if not GMP_API_KEY:
    raise ValueError("GMP_API_KEY is missing. Please set it in Phase 1.")

# Calculate center and bounds dynamically from the dataframe
center_lat = df_scored['lat'].mean()
center_lng = df_scored['lng'].mean()
min_lat, max_lat = df_scored['lat'].min(), df_scored['lat'].max()
min_lng, max_lng = df_scored['lng'].min(), df_scored['lng'].max()
ZOOM_LEVEL = 12

# 2. Create Google Maps Session
print("🗺️ Initializing Google Maps Session...")
session_url = f"https://tile.googleapis.com/v1/createSession?key={GMP_API_KEY}"
headers = {"Content-Type": "application/json"}
payload = {
    "mapType": "roadmap",
    "language": "en-US",
    "region": "US"
}

try:
    response = requests.post(session_url, json=payload, headers=headers)
    response.raise_for_status()
    session_token = response.json().get("session")
    print(f"✅ Session Token acquired: {session_token[:10]}...")
except Exception as e:
    raise RuntimeError(f"Failed to initialize Google Maps session: {e}")

# 3. Fetch Dynamic Attribution
# We use the bounds of our data to request the specific copyright for this area
print("Fetching dynamic attribution...")
viewport_url = (
    f"https://tile.googleapis.com/tile/v1/viewport?key={GMP_API_KEY}"
    f"&session={session_token}"
    f"&zoom={ZOOM_LEVEL}"
    f"&north={max_lat}&south={min_lat}"
    f"&west={min_lng}&east={max_lng}"
)

try:
    vp_response = requests.get(viewport_url)
    vp_response.raise_for_status()
    google_attribution = vp_response.json().get('copyright', 'Map data © Google')
    print(f"✅ Attribution received: {google_attribution}")
except Exception as e:
    print(f"⚠️ Warning: Could not fetch attribution ({e}). Defaulting.")
    google_attribution = "Map data © Google"

# 4. Construct Logo HTML
# Standard Google on-map logo asset
logo_url = "https://maps.gstatic.com/mapfiles/api-3/images/google_white3.png"

logo_html = f"""
    <div style="
        position: fixed;
        bottom: 35px;
        left: 10px;
        z-index: 9999;
        font-size: 0;
        pointer-events: none;
    ">
        <img src="{logo_url}" alt="Google Maps" style="height: 18px;">
    </div>
"""
print("✅ Logo HTML prepared.")

In [ ]:
# @title Phase 4b: Map Visualization
# @markdown Visualizes the **custom location scores** generated by combining **Places Insights** data with **BigQuery `AI.GENERATE`**.
# @markdown
# @markdown This interactive map plots the apartment listings, allowing you to instantly see which locations best fit the target persona and explore the reasoning.

import folium

# 1. Construct Tile URL
# Note: {{z}}, {{x}}, {{y}} are kept as literals for Folium/Leaflet to substitute.
tiles_url = f"https://tile.googleapis.com/v1/2dtiles/{{z}}/{{x}}/{{y}}?session={session_token}&key={GMP_API_KEY}"

# 2. Initialize Map
m = folium.Map(
    location=[center_lat, center_lng],
    zoom_start=ZOOM_LEVEL,
    tiles=tiles_url,
    attr=google_attribution, # Uses the dynamic copyright fetched in Phase 4a
    name="Google Maps",
    control_scale=True,
    prefer_canvas=True
)

# 3. Create Custom Legend HTML
# Positioned at Bottom-Right to avoid the Google Logo at Bottom-Left.
legend_html = '''
     <div style="position: fixed; bottom: 50px; right: 50px; width: 180px; height: 110px;
     border:2px solid grey; z-index:9999; font-size:14px; background-color:white; opacity:0.9; padding: 10px; font-family: sans-serif;">
     <b>AI Score Legend</b><br>
     &nbsp; <i class="glyphicon glyphicon-star" style="color:green"></i> &nbsp; High (8-10)<br>
     &nbsp; <i class="glyphicon glyphicon-info-sign" style="color:orange"></i> &nbsp; Medium (5-7)<br>
     &nbsp; <i class="glyphicon glyphicon-exclamation-sign" style="color:red"></i> &nbsp; Low (0-4)<br>
     </div>
     '''

# 4. Inject Overlays (Logo + Legend)
# We add the logo to the left, and the legend to the right.
m.get_root().html.add_child(folium.Element(logo_html))   # Logo (from Phase 4a)
m.get_root().html.add_child(folium.Element(legend_html)) # Legend

# 5. Add Scored Markers
print("📍 Plotting AI-scored locations...")

for _, row in df_scored.iterrows():
    # Dynamic Color Logic based on AI Score
    score = row['score']
    if score >= 8.0:
        icon_color = 'green'
        icon_type = 'star'
    elif score >= 5.0:
        icon_color = 'orange'
        icon_type = 'info-sign'
    else:
        icon_color = 'red'
        icon_type = 'exclamation-sign'

    # Popup Content (HTML)
    popup_html = f"""
    <div style="font-family: sans-serif; width: 250px;">
        <h4>{row['name']}</h4>
        <p><b>Score:</b> <span style="color:{icon_color}; font-size: 1.2em; font-weight:bold;">{score}/10</span></p>
        <p><i>"{row['reasoning']}"</i></p>
    </div>
    """

    folium.Marker(
        location=[row['lat'], row['lng']],
        popup=folium.Popup(popup_html, max_width=300),
        tooltip=f"{row['name']} ({score})",
        icon=folium.Icon(color=icon_color, icon=icon_type)
    ).add_to(m)

# 6. Display
display(m)

In [ ]:
# @title Phase 5: Clean Up Resources
# @markdown This cell deletes the demo dataset and all tables within it.
# @markdown
# @markdown **You will be prompted to confirm before deletion.**

from google.cloud.exceptions import NotFound

# Validation
print(f"⚠️ WARNING: You are about to DELETE the dataset: `{DATASET_ID}`")
print(f"   Project: `{GCP_PROJECT_ID}`")
print("   This action cannot be undone.")

# Interactive Input
confirmation = input("Type 'yes' to proceed with deletion: ").strip().lower()

if confirmation == 'yes':
    print(f"\n🗑️ Deleting dataset: {DATASET_ID}...")
    try:
        # delete_contents=True removes tables inside the dataset
        client.delete_dataset(DATASET_ID, delete_contents=True, not_found_ok=True)
        print(f"✅ Successfully deleted dataset '{DATASET_ID}' and all contents.")
    except Exception as e:
        print(f"❌ Error deleting dataset: {e}")
else:
    print(f"\n🛑 Operation cancelled. Dataset `{DATASET_ID}` was NOT deleted.")